# Potential Local Authorities (Minor Roads)

## Imports

In [1]:
import urllib.request, json
import pandas as pd
import math
import pickle
import collections
from tqdm import tqdm
import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import statsmodels.api as sm

## Global Variables

In [2]:
COLAB = True

In [3]:
UNWANTED_COLS = ['region_name', 'region_id', 'road_type', 'pedal_cycles', 'two_wheeled_motor_vehicles', 'hgvs_2_rigid_axle',
                    'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
                    'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle',
                    'hgvs_6_articulated_axle', 'start_junction_road_name', 'end_junction_road_name', 'easting',
                    'northing', 'estimation_method', 'estimation_method_detailed', 'link_length_miles']


AADT_LOCAL_AUTHORITY_IDS = [('Luton', '120'), ('Worcestershire', '69'), ('Hounslow', '111')]

ROOT_DIR_PATH = os.path.abspath('..')

if COLAB:

  from google.colab import drive
  drive.mount('/content/drive')

  ROOT_DIR_PATH = os.path.abspath('drive/MyDrive/Spatial_Finance_Transport/minorRoads/')

AADT_DATA_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/aadt/')

GHG_EMISSIONS_DATA_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/ghg_emissions/')

GHG_PROCESSED_PATH = os.path.join(ROOT_DIR_PATH, 'data/ground_truth_data/ghg_emissions/GHG_potential_sites.csv')

LOCAL_AUTHORITY_NAMES = ['Luton', 'Worcester', 'Hounslow', 'Portsmouth', 'Southampton', 'South Tyneside', 'Enfield', 'Halton', 'Barnet', 'Dudley', 'Coventry', 
                         'Trafford', 'Bracknell Forest', 'Havering', 'Sunderland', 'Liverpool', 'Blackburn with Darwen',
                         'Bolton'] 

LOCAL_AUTHORITY_IDS = ['120', '69', '111', '82', '137', '166', '121', '156', '57', '189', '152', '91', '180', 
                       '201', '161', '198', '94', '160']


CHOSEN_COUNT_SITES = [('Luton', 'M1/2557A', 'M1/2557B'), ('Hounslow', 'M4/2188A', 'M4/2188B'), ('Enfield', 'M25/5441A', 'M25/5441B'), ('Blackburn with Darwen', '30361033', '30361032'), ('Havering', 'M25/5790A', 'M25/5790B'),
                      ('Trafford', 'M60/9083A', 'M60/9086B')]

CHOSEN_LA_NAMES = ['Luton', 'Hounslow', 'Enfield', 'Blackburn with Darwen', 'Havering', 'Trafford']

VEHICLE_TYPES = ['cars_and_taxis', 'buses_and_coaches', 'lgvs', 'all_hgvs']

Mounted at /content/drive


## Helper Functions

In [4]:
def clean_aadt_list(df_aadt_list):
    
    for i in range(len(df_aadt_list)):
        df = df_aadt_list[i]
        df_aroads = df[df['road_type'].str.contains('Minor')]
        df_2005_onwards = df_aroads[df_aroads['year'] >= 2005]  
        df_2005_onwards =  df_2005_onwards[df_2005_onwards['year'] != 2021]  
        df_aadt_list[i] = df_2005_onwards.drop(UNWANTED_COLS, axis=1, errors='ignore')

    return df_aadt_list

In [5]:
def group_aadt_list(df_aadt_list):

    grouped_df_aadt_list = []

    for i in range(len(df_aadt_list)):
        df = df_aadt_list[i]
        #grouped_df = df.groupby(by=['count_point_id', 'year']).mean()
        local_authority_name = df.iloc[0]['local_authority_name']
        grouped_df = df.groupby(by=['year']).median()
        grouped_df['Calendar Year'] = grouped_df.index
        grouped_df = grouped_df.reset_index()
        grouped_df['Local Authority'] = local_authority_name
        grouped_df_aadt_list.append(grouped_df)

    return grouped_df_aadt_list

In [6]:
def merge_aadt_ghg_list(df_aadt_list, df_ghg_list):

    merged_dfs_list = []

    for i, aadt_df in enumerate(df_aadt_list):
        for j, ghg_df in enumerate(df_ghg_list):
            if aadt_df.iloc[0]['Local Authority'] == ghg_df.iloc[0]['Local Authority']:
                print("AADT DF len: {}, GHG DF len: {}".format(len(aadt_df), len(ghg_df)))
                merged_df = aadt_df.merge(ghg_df, on=['Calendar Year', 'Local Authority'])
                merged_dfs_list.append(merged_df)

    return merged_dfs_list

In [29]:
def plot_aadt_ghg(merged_dfs_list, show_plot=True, show_ols=True):
    fig = go.Figure()

    colors = px.colors.qualitative.Plotly * 2

    df_ols = merged_dfs_list[0][['Calendar Year']].copy()

    df_total = merged_dfs_list[0][['all_motor_vehicles', 'Annual Territorial emissions (kt CO2e)']].copy()

    for i, df in enumerate(merged_dfs_list):
        la_name = df.iloc[0]['Local Authority']
        fig = fig.add_trace(go.Scatter(x = df['all_motor_vehicles'], y=df['Annual Territorial emissions (kt CO2e)'], mode='markers', name=la_name, marker=dict(color=colors[i])))

        x = sm.add_constant(df['all_motor_vehicles'])
        model = sm.OLS(df['Annual Territorial emissions (kt CO2e)'], x).fit()
        df_ols[la_name+ '_OLS'] = model.fittedvalues

        color = fig.data[0].line.color
        # regression data
        if show_ols:
            fig.add_trace(go.Scatter(x=df['all_motor_vehicles'],
                                y=df_ols[la_name + '_OLS'],
                                mode='lines',
                                name=la_name+' OLS',
                                marker=dict(color=colors[i],size=15)
                                ))
        
        if i > 0:
            df_total = pd.concat([df_total, df[['all_motor_vehicles', 'Annual Territorial emissions (kt CO2e)']]], ignore_index=True)


    df_total = df_total.dropna()

    x = sm.add_constant(df_total['all_motor_vehicles'])
    model = sm.OLS(df_total['Annual Territorial emissions (kt CO2e)'], x).fit()
    df_total['Total_OLS'] = model.fittedvalues

    fig.add_trace(go.Scatter(x=df_total['all_motor_vehicles'],
                            y=df_total['Total_OLS'],
                            mode='lines',
                            name='Total OLS',
                            line=dict(color='black',width=3)
                            ))

    if show_plot:
        fig.update_xaxes(title='AADT')
        fig.update_yaxes(title='Minor Roads: Annual Territorial emissions (kt CO2e)')

        fig.update_layout(
            autosize=False,
            width=1000,
            height=1000,
            legend=dict(font=dict(size=20)))
        
        fig.update_layout(title="Minor Roads: AADT vs GHG Emissions")
        
        fig.show()

    return df_ols, df_total

## AADT Data Loading and Pre-Processing

### Load Data

In [8]:
df_aadt_list = []

for id in LOCAL_AUTHORITY_IDS:
    df_aadt_list.append(pd.read_csv('https://storage.googleapis.com/dft-statistics/road-traffic/downloads/aadfbydirection/local_authority_id/dft_aadfbydirection_local_authority_id_{}.csv'.format(id)))

print(df_aadt_list[0].columns)

print(df_aadt_list[0]['count_point_id'].unique())

print(df_aadt_list[0]['road_type'].unique())

df_aadt_list[0].head()

Index(['count_point_id', 'year', 'region_id', 'region_name',
       'local_authority_id', 'local_authority_name', 'road_name', 'road_type',
       'start_junction_road_name', 'end_junction_road_name', 'easting',
       'northing', 'latitude', 'longitude', 'link_length_km',
       'link_length_miles', 'estimation_method', 'estimation_method_detailed',
       'direction_of_travel', 'pedal_cycles', 'two_wheeled_motor_vehicles',
       'cars_and_taxis', 'buses_and_coaches', 'lgvs', 'hgvs_2_rigid_axle',
       'hgvs_3_rigid_axle', 'hgvs_4_or_more_rigid_axle',
       'hgvs_3_or_4_articulated_axle', 'hgvs_5_articulated_axle',
       'hgvs_6_articulated_axle', 'all_hgvs', 'all_motor_vehicles'],
      dtype='object')
[ 18221  27247  28472   6176  38079  48450  48057  73041  73040  73632
  77396  80935  99950  80936  80937 941719 941721 941688 941702 941713
  81081 941705 941700 941691  81080 941693 941708  81315  81457 951632
  17959 951634  77397 941699 941714 941716  84098  73039  73042  8145

,count_point_id,year,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,...,buses_and_coaches,lgvs,hgvs_2_rigid_axle,hgvs_3_rigid_axle,hgvs_4_or_more_rigid_axle,hgvs_3_or_4_articulated_axle,hgvs_5_articulated_axle,hgvs_6_articulated_axle,all_hgvs,all_motor_vehicles
0,18221,2014,7,East of England,120,Luton,A505,Major,Eaton Green Road roundabout,A5228,...,38,1099,171,57,23,11,14,43,318,8711
1,18221,2014,7,East of England,120,Luton,A505,Major,Eaton Green Road roundabout,A5228,...,35,1091,188,41,37,13,15,48,341,9314
2,27247,2014,7,East of England,120,Luton,A505,Major,Castle St roundabout,A6,...,124,1525,142,22,24,3,18,11,220,14023
3,27247,2014,7,East of England,120,Luton,A505,Major,Castle St roundabout,A6,...,95,1512,142,13,29,8,8,14,215,14091
4,28472,2014,7,East of England,120,Luton,A505,Major,Kimpton Road roundabout,Eaton Green Road roundabout,...,93,1429,118,16,31,10,26,26,228,11015


### Keep only Minor Roads and Remove unwanted Columns

In [9]:
df_aadt_list = clean_aadt_list(df_aadt_list)

df_aadt_list[0]

,count_point_id,year,local_authority_id,local_authority_name,road_name,latitude,longitude,link_length_km,direction_of_travel,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,all_motor_vehicles
28,941719,2014,120,Luton,U,51.904829,-0.476626,NaN,N,220,25,12,3,259
29,941719,2014,120,Luton,U,51.904829,-0.476626,NaN,S,146,1,16,1,164
30,941721,2014,120,Luton,U,51.873062,-0.411431,NaN,W,1154,0,202,17,1379
31,941688,2014,120,Luton,B579,51.895729,-0.440949,NaN,E,5432,113,550,95,6207
32,941688,2014,120,Luton,B579,51.895729,-0.440949,NaN,W,6067,107,595,146,6930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,803821,2019,120,Luton,U,51.919432,-0.436391,NaN,W,6007,39,367,16,6458
1653,803822,2019,120,Luton,U,51.900696,-0.399104,NaN,N,2318,38,231,9,2601
1654,803822,2019,120,Luton,U,51.900696,-0.399104,NaN,S,1624,39,151,9,1828
1657,803825,2019,120,Luton,U,51.895861,-0.457891,NaN,N,4792,35,573,62,5474


In [10]:
for df in df_aadt_list:
    fig = px.histogram(df, x="all_motor_vehicles")
    
fig.show()

### Groupby year and/or count point id

In [11]:
grouped_df_aadt_list = group_aadt_list(df_aadt_list)

grouped_df_aadt_list[0]

<ipython-input-5-daadee7f310a>:9: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-5-daadee7f310a>:9: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-5-daadee7f310a>:9: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

<ipython-input-5-daadee7f310a>:9: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to

,year,count_point_id,local_authority_id,latitude,longitude,link_length_km,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,all_motor_vehicles,Calendar Year,Local Authority
0,2005,930580.0,120.0,51.899671,-0.437625,NaN,4117.0,58.0,376.0,49.0,4602.0,2005,Luton
1,2006,930580.0,120.0,51.899671,-0.437625,NaN,4079.0,57.0,379.0,56.0,4518.0,2006,Luton
2,2007,930580.0,120.0,51.897697,-0.437625,NaN,2940.0,71.0,383.0,63.0,3499.0,2007,Luton
3,2008,941710.0,120.0,51.893508,-0.437625,NaN,847.0,7.0,85.0,22.0,988.0,2008,Luton
4,2009,941704.0,120.0,51.891996,-0.431503,NaN,2414.0,36.0,245.5,25.5,2836.0,2009,Luton
5,2010,941708.0,120.0,51.892028,-0.433947,NaN,305.0,3.0,29.0,5.0,360.0,2010,Luton
6,2011,941708.0,120.0,51.892028,-0.433947,NaN,386.0,2.0,35.0,2.0,432.0,2011,Luton
7,2012,941708.0,120.0,51.892028,-0.433947,NaN,382.0,3.0,37.0,3.0,423.0,2012,Luton
8,2013,941708.0,120.0,51.892028,-0.433947,NaN,351.0,2.0,44.0,4.0,406.0,2013,Luton
9,2014,941706.5,120.0,51.892986,-0.437448,NaN,333.0,2.0,35.5,3.0,377.0,2014,Luton


## GHG Emissions Data Loading and Pre-Processing

In [12]:
df_ghg = pd.read_csv(GHG_PROCESSED_PATH, index_col=0)

df_ghg

,Local Authority,Calendar Year,Annual Territorial emissions (kt CO2e)
128,Barnet,2005,113.556626
129,Barnet,2006,107.078941
130,Barnet,2007,107.428381
131,Barnet,2008,102.377640
132,Barnet,2009,99.827042
...,...,...,...
4859,Worcester,2016,62.312940
4860,Worcester,2017,61.993240
4861,Worcester,2018,62.813507
4862,Worcester,2019,61.146978


In [13]:
df_ghg = df_ghg[df_ghg['Local Authority'].isin(LOCAL_AUTHORITY_NAMES)]

df_ghg_list = [d for _, d in df_ghg.groupby(['Local Authority'])]

len(df_ghg_list)

<ipython-input-13-b6a3c2695393>:3: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



18

### AADT and GHG Emissions

In [14]:
merged_dfs_list = merge_aadt_ghg_list(grouped_df_aadt_list, df_ghg_list)

merged_dfs_list[0]

AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16
AADT DF len: 16, GHG DF len: 16


,year,count_point_id,local_authority_id,latitude,longitude,link_length_km,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,all_motor_vehicles,Calendar Year,Local Authority,Annual Territorial emissions (kt CO2e)
0,2005,930580.0,120.0,51.899671,-0.437625,NaN,4117.0,58.0,376.0,49.0,4602.0,2005,Luton,147.206665
1,2006,930580.0,120.0,51.899671,-0.437625,NaN,4079.0,57.0,379.0,56.0,4518.0,2006,Luton,142.744281
2,2007,930580.0,120.0,51.897697,-0.437625,NaN,2940.0,71.0,383.0,63.0,3499.0,2007,Luton,146.037257
3,2008,941710.0,120.0,51.893508,-0.437625,NaN,847.0,7.0,85.0,22.0,988.0,2008,Luton,140.888523
4,2009,941704.0,120.0,51.891996,-0.431503,NaN,2414.0,36.0,245.5,25.5,2836.0,2009,Luton,137.339401
5,2010,941708.0,120.0,51.892028,-0.433947,NaN,305.0,3.0,29.0,5.0,360.0,2010,Luton,134.214543
6,2011,941708.0,120.0,51.892028,-0.433947,NaN,386.0,2.0,35.0,2.0,432.0,2011,Luton,133.123099
7,2012,941708.0,120.0,51.892028,-0.433947,NaN,382.0,3.0,37.0,3.0,423.0,2012,Luton,128.707920
8,2013,941708.0,120.0,51.892028,-0.433947,NaN,351.0,2.0,44.0,4.0,406.0,2013,Luton,127.337597
9,2014,941706.5,120.0,51.892986,-0.437448,NaN,333.0,2.0,35.5,3.0,377.0,2014,Luton,129.615845


## Visualisation

### AADT

In [28]:
fig = go.Figure()

for i, df in enumerate(grouped_df_aadt_list):
    fig = fig.add_trace(go.Scatter(x = df['Calendar Year'], y=df['all_motor_vehicles'], name=df.iloc[0]['Local Authority']))

fig.update_xaxes(title='Year')
fig.update_yaxes(title='AADT')
fig.update_layout(title="Minor Roads: AADT Per Year ")

fig.show()

### GHG Emissions

In [27]:
fig = go.Figure()

for i, df in enumerate(df_ghg_list):
    fig = fig.add_trace(go.Scatter(x = df['Calendar Year'], y=df['Annual Territorial emissions (kt CO2e)'], name=df.iloc[0]['Local Authority']))

fig.update_xaxes(title='Year')
fig.update_yaxes(title='Annual Territorial emissions (kt CO2e)')
fig.update_layout(title="Minor Roads: GHG Emissions Per Year ")

fig.show()

### AADT and GHG Emissions

In [30]:
df_ols, df_total = plot_aadt_ghg(merged_dfs_list, show_plot=True, show_ols=False)

## Filter for Sites with Good Count Data

In [18]:
chosen_site_names = [item[0] for item in CHOSEN_COUNT_SITES]

chosen_merged_dfs_list = []

for df in merged_dfs_list:
    if df.iloc[0]['Local Authority'] in chosen_site_names:
        chosen_merged_dfs_list.append(df)

In [19]:
df_ols, df_total = plot_aadt_ghg(chosen_merged_dfs_list)

### Count Site AADT Distributions

In [20]:
df_chosen_aadt_list = []

for name in CHOSEN_LA_NAMES:
    for df in df_aadt_list:
        if df.iloc[0]['local_authority_name'] == name:
            df_chosen_aadt_list.append(df)

df_chosen_aadt_list[0]

,count_point_id,year,local_authority_id,local_authority_name,road_name,latitude,longitude,link_length_km,direction_of_travel,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,all_motor_vehicles
28,941719,2014,120,Luton,U,51.904829,-0.476626,NaN,N,220,25,12,3,259
29,941719,2014,120,Luton,U,51.904829,-0.476626,NaN,S,146,1,16,1,164
30,941721,2014,120,Luton,U,51.873062,-0.411431,NaN,W,1154,0,202,17,1379
31,941688,2014,120,Luton,B579,51.895729,-0.440949,NaN,E,5432,113,550,95,6207
32,941688,2014,120,Luton,B579,51.895729,-0.440949,NaN,W,6067,107,595,146,6930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,803821,2019,120,Luton,U,51.919432,-0.436391,NaN,W,6007,39,367,16,6458
1653,803822,2019,120,Luton,U,51.900696,-0.399104,NaN,N,2318,38,231,9,2601
1654,803822,2019,120,Luton,U,51.900696,-0.399104,NaN,S,1624,39,151,9,1828
1657,803825,2019,120,Luton,U,51.895861,-0.457891,NaN,N,4792,35,573,62,5474


In [21]:
for df in df_chosen_aadt_list:
    fig = px.histogram(df, x="all_motor_vehicles", title=df.iloc[0]['local_authority_name'])
    fig.update_layout(
            autosize=False,
            width=750,
            height=500,
            font=dict(size=20),
            )
        
    fig.show()

### Count Site Vehicle Type Distribution

In [22]:
colors = ['blue', 'green', 'red', 'orange']

for df in df_chosen_aadt_list:
    # Group the DataFrame by year and calculate median for each column
    df_grouped = df.groupby('year').median(numeric_only=True).reset_index()

    fig = go.Figure()

    for i, vehicle_type in enumerate(VEHICLE_TYPES):
        fig.add_trace(go.Bar(
            x=df_grouped['year'],
            y=df_grouped[vehicle_type],
            name=vehicle_type.replace('_', ' ').title(),
            marker=dict(color=colors[i])
        ))

        # Calculate percentage makeup within each year
        total_per_year = df_grouped[VEHICLE_TYPES].sum(axis=1)
        pct_makeup = df_grouped[vehicle_type] / total_per_year * 100
        
        # Calculate the cumulative sum of previous vehicle types' values
        cumulative_sum = df_grouped[VEHICLE_TYPES[:i]].sum(axis=1)
        
        # Add text annotations for percentage values
        fig.add_trace(go.Scatter(
            x=df_grouped['year'],
            y=cumulative_sum + (0.5 * df_grouped[vehicle_type]),
            mode='text',
            text=[f'{val:.1f}%' for val in pct_makeup],
            textposition='middle center',
            textfont=dict(color='white', size=10),
            cliponaxis=False
        ))

    fig.update_layout(
        barmode='stack',
        title=df.iloc[0]['local_authority_name']+" Median Motor Vehicles by Year",
        autosize=False,
        width=1000,
        height=800,
        font=dict(size=20),
        yaxis=dict(title='Median Value')
    )

    fig.show()

## Final Sites EDA

In [23]:
chosen_merged_dfs_list[0]

,year,count_point_id,local_authority_id,latitude,longitude,link_length_km,cars_and_taxis,buses_and_coaches,lgvs,all_hgvs,all_motor_vehicles,Calendar Year,Local Authority,Annual Territorial emissions (kt CO2e)
0,2005,930580.0,120.0,51.899671,-0.437625,NaN,4117.0,58.0,376.0,49.0,4602.0,2005,Luton,147.206665
1,2006,930580.0,120.0,51.899671,-0.437625,NaN,4079.0,57.0,379.0,56.0,4518.0,2006,Luton,142.744281
2,2007,930580.0,120.0,51.897697,-0.437625,NaN,2940.0,71.0,383.0,63.0,3499.0,2007,Luton,146.037257
3,2008,941710.0,120.0,51.893508,-0.437625,NaN,847.0,7.0,85.0,22.0,988.0,2008,Luton,140.888523
4,2009,941704.0,120.0,51.891996,-0.431503,NaN,2414.0,36.0,245.5,25.5,2836.0,2009,Luton,137.339401
5,2010,941708.0,120.0,51.892028,-0.433947,NaN,305.0,3.0,29.0,5.0,360.0,2010,Luton,134.214543
6,2011,941708.0,120.0,51.892028,-0.433947,NaN,386.0,2.0,35.0,2.0,432.0,2011,Luton,133.123099
7,2012,941708.0,120.0,51.892028,-0.433947,NaN,382.0,3.0,37.0,3.0,423.0,2012,Luton,128.707920
8,2013,941708.0,120.0,51.892028,-0.433947,NaN,351.0,2.0,44.0,4.0,406.0,2013,Luton,127.337597
9,2014,941706.5,120.0,51.892986,-0.437448,NaN,333.0,2.0,35.5,3.0,377.0,2014,Luton,129.615845


## Saving Data

In [24]:
print("number of chosen LA's: {}".format(len(chosen_merged_dfs_list)))

for df in chosen_merged_dfs_list:
    df_la_aadt = df[['year', 'cars_and_taxis', 'buses_and_coaches',	'lgvs', 'all_hgvs', 'all_motor_vehicles', 'Local Authority']]
    df_la_ghg = df[['year', 'Annual Territorial emissions (kt CO2e)', 'Local Authority']]

    df_la_aadt.name = 'all_motor_vehicles_'+df_la_aadt.iloc[0]['Local Authority']
    df_la_ghg.name = 'ghg_emissions_'+df_la_ghg.iloc[0]['Local Authority']

    df_la_aadt.to_csv(AADT_DATA_PATH+df_la_aadt.name+'.csv')
    df_la_ghg.to_csv(GHG_EMISSIONS_DATA_PATH+df_la_ghg.name+'.csv')

number of chosen LA's: 6
